# Test GPT in Python

In [29]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from openai import OpenAI

## Import Data

In [30]:
df = pd.read_csv("./sst_dataset/SST2-Data/SST2-Data/stanfordSentimentTreebank/stanfordSentimentTreebank/datasetSentences.txt", sep="\t", index_col=0)
datasetSplit = pd.read_csv("./sst_dataset/SST2-Data/SST2-Data/stanfordSentimentTreebank/stanfordSentimentTreebank/datasetSplit.txt", index_col=0)
df_test = df[datasetSplit['splitset_label'] == 2]

## gpt-3.5-turbo classification

In [59]:
client = OpenAI(
    api_key="api-key"
)

def classify_sentiment_zero_shot(sentence):
    #prompt =    f"""Classify the sentiment of this sentence into one of the following categories: 
    #            very negative, negative, neutral, positive, very positive. Only answer with 0 for 
    #            very negative, 1 for negative, 2 for neutral, 3 for positive or 4 for very 
    #            positive. Sentence: '{sentence}'"""
    
    prompt =    f"""Examples of sentiment classification:\n
                Sentence: ... a sour little movie at its core ; an exploration of the emptiness that underlay the relentless gaiety of the 1920 's ... The film 's ending has a `` What was it all for ? '' -> Class: very negative\n
                Sentence: As it is , it 's too long and unfocused . -> Class: very negative\n
                Sentence: This is n't a new idea . -> Class: negative\n
                Sentence: Made me unintentionally famous -- as the queasy-stomached critic who staggered from the theater and blacked out in the lobby . -> Class: negative\n
                Sentence: You 'd think by now America would have had enough of plucky British eccentrics with hearts of gold . -> Class: neutral\n
                Sentence: You Should Pay Nine Bucks for This : Because you can hear about suffering Afghan refugees on the news and still be unaffected . -> Class: neutral\n
                Sentence: The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal . -> Class: positive\n
                Sentence: Singer\/composer Bryan Adams contributes a slew of songs -- a few potential hits , a few more simply intrusive to the story -- but the whole package certainly captures the intended , er , spirit of the piece . -> Class: positive\n
                Sentence: The gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a column of words can not adequately describe co-writer\/director Peter Jackson 's expanded vision of J.R.R. Tolkien 's Middle-earth . -> Class: very positive\n
                Sentence: Whether or not you 're enlightened by any of Derrida 's lectures on `` the other '' and `` the self , '' Derrida is an undeniably fascinating and playful fellow . -> Class: very positive\n
                According to the above examples classify the sentiment of the following sentence into one of these categories: very negative (0), negative (1), neutral (2), positive (3), very positive (4). Respond only with the number. Do not include any additional text or explanation in your response.\n
                Sentence: '{sentence}'"""

    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role":"user",
                "content":prompt
            },    
        ],
        model="gpt-3.5-turbo",
        max_tokens=1,
        seed=42
    )

    return chat_completion.choices[0].message.content

In [66]:
#################################################################################################################
# ATTENTION! This line costs money!

#predicted_labels_list = [classify_sentiment_zero_shot(sentence) for sentence in df_test['sentence']]

# ATTENTION! This line costs money!
#################################################################################################################

### Create dataframe of predicted labels

In [67]:
predicted_labels_df = pd.DataFrame(predicted_labels_list, index=df_test.index, columns=['classified_sentiment'])
predicted_labels_df['classified_sentiment'] = pd.to_numeric(predicted_labels_df['classified_sentiment'], errors='coerce')
predicted_labels_df.reset_index(drop=True, inplace=True)
# For 'Wrong answers' that are not convertable to int
nan_count = predicted_labels_df.isna().sum().sum()
print(nan_count)
predicted_labels_df

15


,classified_sentiment
0,1.0
1,3.0
2,3.0
3,3.0
4,3.0
...,...
2205,3.0
2206,4.0
2207,3.0
2208,4.0


## Write predicted labels to file

In [70]:
predicted_labels_df.to_csv('./sst_dataset/SST2-Data/SST2-Data/stanfordSentimentTreebank/stanfordSentimentTreebank/predicted_sentiment_labels_few_shot.txt', sep='\t', index=True)